In [98]:
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pandas as pd
import matplotlib.pyplot as plt
import itertools
from itertools import combinations
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from statsmodels.stats.anova import anova_lm


In [26]:
AC15 = pd.read_csv('AC15.csv', delimiter=',')
y = np.array(AC15['LOGTN'])
AC15 = AC15.drop('LOGTN', axis = 1)
AC15.columns

Index(['LOGCA', 'LOGIMP', 'MMJTEMP', 'MSRAIN', 'PRES', 'PNON', 'PCOMM',
       'PIND'],
      dtype='object')

### PRESS

In [ ]:
def press():
    y = np.array(AC15['LOGTN'])
    for column in AC15: #dependant variable 
        X1 = np.array(AC15[column])
        X2 = np.array(AC15[column])
        combination = np.column_stack((X1, X2))
        X_train, X_test, Y_train, Y_test = train_test_split(combination, y test_size=0.2, random_state=42)
        model = LinearRegression()
        model.fit(X_train, y)
        y_pred = model.predict(X_test)
        print(' residual sum of squares for', column, 'is : '+ str(np.sum(np.square(y_pred - y))))

In [36]:


columns = AC15.columns
names_final = []
rss_final = []
for i in range(1, len(columns)+1):
    combinations = itertools.combinations(columns, i)
    for combination in combinations:
        # Create a new DataFrame with the selected columns
        subset_df = AC15[list(combination)]
        matrix = subset_df.to_numpy()

        # print(f"Combination: {combination}")
        X_train, X_test, Y_train, Y_test = train_test_split(matrix, y, test_size=0.2, random_state=42)
        # print(X_train)
        model = LinearRegression()
        model.fit(X_train, Y_train)
        y_pred = model.predict(X_test)
        rss = np.sum(np.square(y_pred - Y_test))
        names_final.append(combination)
        rss_final.append(rss)
        # print(' residual sum of squares for', combination, 'is : '+ str(np.sum(np.square(y_pred - Y_test))))
        # print(matrix)

In [61]:
combinations_df = pd.DataFrame(rss_final, names_final)
min = combinations_df.min(axis=0)
print('Lowest residual sum of squares is:', min)
best_model = combinations_df[combinations_df[0] < 1.49]
best_model

Lowet residual sum of squares is: 0    1.478086
dtype: float64


,0
"(LOGCA, LOGIMP, PRES, PIND)",1.478086


### ANOVA (First output is just for linear, the second is every combo)

In [96]:
AC15 = pd.read_csv('AC15.csv', delimiter=',')
formula = "LOGTN ~ LOGCA + LOGIMP + MMJTEMP + MSRAIN + PRES + PNON + PCOMM + PIND"
model = smf.ols(formula=formula, data=AC15).fit()
anova_results = sm.stats.anova_lm(model, typ=2)
print(anova_results)


             sum_sq    df          F    PR(>F)
LOGCA      9.037339   1.0  24.401890  0.000022
LOGIMP     0.854484   1.0   2.307208  0.138299
MMJTEMP    1.016542   1.0   2.744784  0.107051
MSRAIN     2.004454   1.0   5.412265  0.026283
PRES       0.009888   1.0   0.026698  0.871204
PNON       0.007690   1.0   0.020763  0.886304
PCOMM      0.008500   1.0   0.022952  0.880505
PIND       0.003784   1.0   0.010216  0.920102
Residual  12.221684  33.0        NaN       NaN


In [101]:
AC15 = pd.read_csv('AC15.csv', delimiter=',')

predictors = ['LOGCA', 'LOGIMP', 'MMJTEMP', 'MSRAIN', 'PRES', 'PNON', 'PCOMM', 'PIND']
response = 'LOGTN'
residuals_final = []
combo_names = []
for r in range(1, len(predictors) + 1):
    for combo in combinations(predictors, r):
        formula = f"{response} ~ {' + '.join(combo)}"
        model = smf.ols(formula=formula, data=AC15).fit()
        anova_results = sm.stats.anova_lm(model, typ=2)
        print(f"\nFormula: {formula}")
        # residuals = model.resid
        # combo_names.append(combo)
        # residuals_final
        print(anova_results)


Formula: LOGTN ~ LOGCA
             sum_sq    df          F    PR(>F)
LOGCA      9.562420   1.0  18.788409  0.000096
Residual  20.358125  40.0        NaN       NaN

Formula: LOGTN ~ LOGIMP
             sum_sq    df         F    PR(>F)
LOGIMP     0.099403   1.0  0.133332  0.716926
Residual  29.821142  40.0       NaN       NaN

Formula: LOGTN ~ MMJTEMP
             sum_sq    df        F    PR(>F)
MMJTEMP    1.254697   1.0  1.75079  0.193292
Residual  28.665848  40.0      NaN       NaN

Formula: LOGTN ~ MSRAIN
             sum_sq    df        F    PR(>F)
MSRAIN     2.013891   1.0  2.88661  0.097085
Residual  27.906654  40.0      NaN       NaN

Formula: LOGTN ~ PRES
             sum_sq    df         F   PR(>F)
PRES       2.586784   1.0  3.785478  0.05875
Residual  27.333761  40.0       NaN      NaN

Formula: LOGTN ~ PNON
             sum_sq    df         F    PR(>F)
PNON       0.053230   1.0  0.071288  0.790842
Residual  29.867315  40.0       NaN       NaN

Formula: LOGTN ~ PCOMM
        